In [1]:
# for auto reload
%load_ext autoreload
%autoreload 2

from RL.hyperparameters.hyperparameters import *
from RL import wandb_utils
import gym
from stable_baselines3 import DDPG
import pickle
from RL import training_utils
import numpy as np
from Data import data_utils

## create environemnt

In [2]:
# create gym-environemnt
hyperparameters = hyper_parameters_dict
env = gym.make(hyperparameters["env_name"], hyper_parameters_dict=hyperparameters)

In [3]:
# get observations from the environment
obs = env.reset()
not_normalized_obs = env.state_to_real_values(obs)
# print it, timeback is the time in history
for i, val in enumerate(obs):
    print(f"normalized value of {env.obs_to_param[i]} in the state observation is {val}")
    print(f"real value of {env.obs_to_param[i]} in the state observation is {not_normalized_obs[i]}")

normalized value of velocity_timeback0 in the state observation is 0.7673197680990869
real value of velocity_timeback0 in the state observation is 15.346395361981738
normalized value of velocity_timeback1 in the state observation is 0.73756478166948
real value of velocity_timeback1 in the state observation is 14.751295633389601
normalized value of velocity_timeback2 in the state observation is 0.7062176025659401
real value of velocity_timeback2 in the state observation is 14.124352051318802
normalized value of velocity_timeback3 in the state observation is 0.686067338207853
real value of velocity_timeback3 in the state observation is 13.721346764157062
normalized value of velocity_timeback4 in the state observation is 0.6754661534821981
real value of velocity_timeback4 in the state observation is 13.509323069643962
normalized value of velocity_timeback5 in the state observation is 0.6529835024289758
real value of velocity_timeback5 in the state observation is 13.059670048579516
normali

In [4]:
# see the possible actions
print(f"the possible actions: {env.action_to_param}")
# see the wind
print(f"wind in simulation position=(0,0,500), time=0: {env.rl_glider_simulation.thermal.get_wind_vel([0,0,500], 0)}")

the possible actions: {0: 'bank_angle', 1: 'attack_angle'}
wind in simulation position=(0,0,500), time=0: [3.        0.        2.2836069]


## run simple algorithm on environment

In [5]:
obs = env.reset()
done = False
while not done:
    action = np.array([1,0])  # make the bank angle high
    obs, rewards, done, info = env.step(action)

In [6]:
# show the plot of this technique
env.render()

## run trained model

In [7]:
# load the model and the config that saved during the training process
with open('hyperparameters_config.pickle', 'rb') as handle:
    config = pickle.load(handle)

hyper_param_dict = wandb_utils.get_hyper_param_dict_from_config(config)
env = gym.make(hyper_param_dict["env_name"], hyper_parameters_dict=hyper_param_dict)
model = DDPG.load("expert_agent.zip")

c:\users\yoavf\appdata\local\programs\python\python38\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning:

Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.



In [8]:
# run the model on the env for 5 trajectories
df = training_utils.get_df_for_analysis_by_model_env(model, env, episodes=5)

100%|██████████| 5/5 [00:11<00:00,  2.37s/it]


In [9]:
# see some statistics on the route
print(f"the mean vz of our agent on 5 trajectories is: {np.mean(df['info_vz_timeback0'])}")

the mean vz of our agent on 5 trajectories is: 0.2954418650538149


In [10]:
# draw one route
data_utils.draw_glider_route(df[df["route_num"]==0]).show()